This is the Melbourne Housing notebook 

In [26]:
import numpy as np 
import pandas as pd 

import sklearn
import fastai

from pathlib import Path
from DS_Utils import *
import seaborn as sns
import matplotlib.pylot as plt



ModuleNotFoundError: No module named 'seaborn'

In [ ]:
path = Path('data')
print(path/'test.csv')

In [ ]:
testdf = pd.read_csv(path/'test.csv', index_col='Id')
traindf = pd.read_csv(path/'train.csv', index_col='Id')

In [ ]:
display_all(traindf.head().T)

In [ ]:
traindf.info()

In [ ]:
missing  = 